In [131]:
import random
from pprint import pprint

import numpy as np

from utils import read_data
from constants import FILENAME_EXAMPLE_1

In [2]:
input_data = read_data(FILENAME_EXAMPLE_1)

days    = input_data['length_of_schedule']
ne      = input_data['number_of_employees']
ns      = input_data['number_of_shifts']
demand  = input_data['temporal_requirements_matrix']
sn      = input_data['shift_name']
ss      = input_data['start_shift']
ls      = input_data['length_shift']
min_ls  = input_data['min_length_of_blocks']
max_ls  = input_data['max_length_of_blocks']
min_do  = input_data['min_days_off']
max_do  = input_data['max_days_off']
min_lw  = input_data['min_length_work_blocks']
max_lw  = input_data['max_length_work_blocks']
nf2     = input_data['nr_sequences_of_length_2']
nf3     = input_data['nr_sequences_of_length_3']
f2      = input_data['not_allowed_shift_sequences_2']
f3      = input_data['not_allowed_shift_sequences_3']

In [3]:
shifts = ns + 1
day, afternoon, night, dayoff = 1, 2, 3, 4
code = sn + ['-']

In [79]:
code

['D', 'A', 'N', '-']

In [17]:
pprint(demand)

[[2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 3, 3, 3, 2], [2, 2, 2, 2, 2, 2, 2]]


- demand[shift][day_int_week]
- 

In [13]:
help(pprint)

Help on function pprint in module pprint:

pprint(object, stream=None, indent=1, width=80, depth=None, *, compact=False, sort_dicts=True)
    Pretty-print a Python object to a stream [default is sys.stdout].



In [143]:
def demand_constraint(result):
    demand_t = np.transpose(demand)
    demand_constraint_result = result
    # number of employees
    # ne = 9
    count_shift = [0]*3
    for shift in range(ns):
        sum(e in EMPLOYEE)(x[e,d] == day) == demand[1, d] /\
        if sum()
        print(demand[shift])
    return result

In [144]:
result = calculate_result(input_data)
demand_constraint(result)

[2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 3, 3, 3, 2]
[2, 2, 2, 2, 2, 2, 2]


[['D', 'D', 'A', '-', 'A', 'D', 'N'],
 ['D', 'D', 'A', '-', 'A', 'D', 'N'],
 ['D', 'D', 'A', '-', 'A', 'D', 'N'],
 ['D', 'D', 'A', '-', 'A', 'D', 'N'],
 ['D', 'D', 'A', '-', 'A', 'D', 'N'],
 ['D', 'D', 'A', '-', 'A', 'D', 'N'],
 ['D', 'D', 'A', '-', 'A', 'D', 'N'],
 ['D', 'D', 'A', '-', 'A', 'D', 'N'],
 ['D', 'D', 'A', '-', 'A', 'D', 'N']]

In [81]:
def calculate_result(input_data):
    result = [['-']*days]*ne

    for e in range(ne):
        for d in range(days):
            result[e][d] = random.choice(code) 

    #result = demand_constraint(result)

    return result

In [9]:
calculate_result(input_data)

[['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A']]